In [2]:
pip install praw


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import praw

In [5]:
reddit = praw.Reddit(
    client_id="",
    client_secret="",
    password="",
    user_agent="",
    username="",
)

In [16]:
import praw
import pandas as pd
from datetime import datetime, timedelta
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName('RedditData').getOrCreate()

subreddit_name = 'nyc'  # Change this to your target subreddit
search_query = 'event'  # Modify this based on how events are typically posted
six_months_ago = datetime.utcnow() - timedelta(days=6*30)  # Approximation of 6 months

# Create a list to store event data
events = []

# Search the subreddit for posts containing 'event' in the title
for submission in reddit.subreddit(subreddit_name).search(search_query, limit=10):  # Adjust the limit as needed
    # if datetime.utcfromtimestamp(submission.created_utc) > six_months_ago:
        submission.comments.replace_more(limit=None)  # Load all comments
        comments = [comment.body for comment in submission.comments.list()]
        event_info = (
            submission.title,
            submission.url,
            datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
            len(comments),
            comments
        )
        events.append(event_info)

# Convert to Pandas DataFrame
columns = ['title', 'url', 'created', 'num_comments', 'comments']
df = pd.DataFrame(events, columns=columns)

# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)
spark_df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/14 16:47:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/envs/bigdata/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


+--------------------+--------------------+-------------------+------------+--------------------+
|               title|                 url|            created|num_comments|            comments|
+--------------------+--------------------+-------------------+------------+--------------------+
|Rockefeller Cente...|https://www.reddi...|2023-11-30 05:50:02|         634|[I swear I feel l...|
|NYC Drag Story Ho...|https://www.nbcne...|2022-12-19 14:52:13|         500|[why is there nev...|
|Drama at a drag q...|https://v.redd.it...|2022-12-12 07:28:34|         474|[So apparently th...|
|Eric Adams attend...|https://www.polit...|2023-11-17 13:41:16|          87|[I just do that o...|
|NYC Mayor Adams i...|https://www.nydai...|2023-05-12 18:19:26|         144|[CUNY Law invited...|
|NYC's Newest Park...|https://i.redd.it...|2021-05-21 17:03:37|         194|[I had no idea th...|
|Republican Jewish...|https://www.haare...|2022-12-27 17:07:57|         103|[This guy is lite...|
|Trump Attends UFC..

In [20]:
import praw
import pandas as pd
from datetime import datetime, timedelta
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName('RedditData').getOrCreate()



subreddit_name = 'nyc'  # Change this to your target subreddit
search_query = 'event'  # Modify this based on how events are typically posted
six_months_ago = datetime.utcnow() - timedelta(days=6*30)  # Approximation of 6 months

# Lists to store event and comments data
events = []
comments_data = []

# Search the subreddit for posts containing 'event' in the title
for submission in reddit.subreddit(subreddit_name).search(search_query, limit=10):  # Adjust the limit as needed
    # if datetime.utcfromtimestamp(submission.created_utc) > six_months_ago:
    submission.comments.replace_more(limit=None)  # Load all comments
    event_info = (
        submission.title,
        submission.url,
        datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
        len(submission.comments)
    )
    events.append(event_info)

    # Extract comments and add to comments_data
    for comment in submission.comments.list():
        comment_info = (submission.title, submission.url, comment.body)
        comments_data.append(comment_info)

# Create DataFrames
events_df = pd.DataFrame(events, columns=['title', 'url', 'created', 'num_comments'])
comments_df = pd.DataFrame(comments_data, columns=['title', 'url', 'comment'])

# Convert Pandas DataFrames to Spark DataFrames
spark_events_df = spark.createDataFrame(events_df)
spark_comments_df = spark.createDataFrame(comments_df)

# Display the DataFrames
spark_events_df.show()
spark_comments_df.show(1000)


+--------------------+--------------------+-------------------+------------+
|               title|                 url|            created|num_comments|
+--------------------+--------------------+-------------------+------------+
|Rockefeller Cente...|https://www.reddi...|2023-11-30 05:50:02|          58|
|NYC Drag Story Ho...|https://www.nbcne...|2022-12-19 14:52:13|          47|
|Drama at a drag q...|https://v.redd.it...|2022-12-12 07:28:34|          73|
|Eric Adams attend...|https://www.polit...|2023-11-17 13:41:16|          27|
|NYC Mayor Adams i...|https://www.nydai...|2023-05-12 18:19:26|          28|
|NYC's Newest Park...|https://i.redd.it...|2021-05-21 17:03:37|          40|
|Republican Jewish...|https://www.haare...|2022-12-27 17:07:57|          35|
|Trump Attends UFC...|https://www.theda...|2019-11-03 12:41:48|          30|
|Ahead of potentia...|https://www.polit...|2023-02-19 17:28:21|          29|
|'Change in percep...|https://www.nbcne...|2021-12-18 16:10:25|          30|